## Football Team Nemesis

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_results = pd.read_csv('../data/results_per_team_and_opponent.csv')
df_aggregated = pd.read_csv('../data/aggregated_result_per_team_and_opponent.csv')

get the worst performing opponent for each team

In [3]:
# get the worst performing opponent for each team
team = 'Milan'
temp = df_aggregated[(df_aggregated['total_num_games'] > 300) & (df_aggregated['team'] == team) & (df_aggregated['num_games'] > 10)].sort_values(by='result_metric')
temp.loc[temp['result_metric'].idxmin(),'opponent']

'Verona'

create dataframe nemesis

In [3]:
# create df nemesis
df_nemesis = pd.DataFrame(columns = ['team', 'nemesis', 'worst_result', 'num_games'])

# here i filter only for teams that have at least 300 games in the dataset
for team in df_aggregated[df_aggregated['total_num_games'] > 300]['team'].unique():
    temp = df_aggregated[(df_aggregated['total_num_games'] > 300) & (df_aggregated['team'] == team) & (df_aggregated['num_games'] > 10)].sort_values(by='sum_points')
    nemesis = temp.loc[temp['pct_points'].idxmin(), 'opponent']
    num_games = temp.loc[temp['pct_points'].idxmin(), 'num_games']
    df_nemesis.loc[len(df_nemesis)] = [team, nemesis, temp['pct_points'].min(), num_games] 

In [4]:
df_nemesis['worst_result'] = round(df_nemesis['worst_result'],2)
df_nemesis.rename(columns={'worst_result':'pct_points'}, inplace=True)

In [5]:
df_nemesis.sort_values(by='pct_points', ascending=False)

,team,nemesis,pct_points,num_games
9,Juventus,Napoli,52.22,30.0
12,Napoli,Empoli,41.67,12.0
11,Milan,Juventus,31.25,32.0
8,Inter,Juventus,28.12,32.0
15,Roma,Juventus,26.04,32.0
13,Palermo,Inter,25.76,22.0
6,Fiorentina,Roma,25.49,34.0
19,Udinese,Juventus,22.92,32.0
7,Genoa,Roma,21.11,30.0
17,Sassuolo,Juventus,16.67,18.0


In [6]:
df_nemesis.to_csv('../data/team_nemesis.csv')

In [8]:
team = 'Milan'

temp = df_aggregated[(df_aggregated['total_num_games'] > 300) &
                     (df_aggregated['team']==team) & (df_aggregated['num_games'] > 10)].sort_values(by='result_metric')
temp.sort_values(by = 'pct_points')

,Unnamed: 0,team,total_num_games,opponent,result_metric,num_games,pct_of_points
695,695,Milan,646.0,Juventus,30.0,32.0,31.250000
694,694,Milan,646.0,Inter,34.0,34.0,33.333333
700,700,Milan,646.0,Napoli,32.0,30.0,35.555556
706,706,Milan,646.0,Roma,43.0,34.0,42.156863
679,679,Milan,646.0,Atalanta,46.0,30.0,51.111111
715,715,Milan,646.0,Udinese,57.0,34.0,55.882353
691,691,Milan,646.0,Fiorentina,57.0,34.0,55.882353
708,708,Milan,646.0,Sampdoria,56.0,32.0,58.333333
696,696,Milan,646.0,Lazio,60.0,34.0,58.823529
709,709,Milan,646.0,Sassuolo,32.0,18.0,59.259259


In [7]:
df_aggregated.columns

Index(['Unnamed: 0', 'team', 'total_num_games', 'opponent', 'sum_points',
       'num_games', 'pct_points'],
      dtype='object')

In [8]:
# create df nemesis complete
df_nemesis_all = df_aggregated.copy()

df_nemesis_all = df_nemesis_all[(df_nemesis_all['total_num_games'] > 300) &
                     (df_aggregated['num_games'] > 10)].sort_values(by='sum_points')
# df_nemesis_all = df_nemesis_all.sort_values(by = 'pct_of_points')
df_nemesis_all = df_nemesis_all.sort_values(by = 'team')

In [9]:
df_nemesis_all

,Unnamed: 0,team,total_num_games,opponent,sum_points,num_games,pct_points
54,54,Atalanta,570.0,Siena,22.0,12.0,61.111111
29,29,Atalanta,570.0,Catania,16.0,14.0,38.095238
25,25,Atalanta,570.0,Bologna,47.0,24.0,65.277778
38,38,Atalanta,570.0,Juventus,14.0,28.0,16.666667
33,33,Atalanta,570.0,Empoli,23.0,14.0,54.761905
...,...,...,...,...,...,...,...
1165,1165,Udinese,646.0,Chievo,40.0,26.0,51.282051
1186,1186,Udinese,646.0,Sampdoria,40.0,32.0,41.666667
1194,1194,Udinese,646.0,Verona,20.0,14.0,47.619048
1159,1159,Udinese,646.0,Bologna,43.0,26.0,55.128205


In [10]:
df_nemesis_all.columns

Index(['Unnamed: 0', 'team', 'total_num_games', 'opponent', 'sum_points',
       'num_games', 'pct_points'],
      dtype='object')

In [11]:
df_nemesis_all = df_nemesis_all[['team', 'total_num_games', 'opponent', 'sum_points',
       'num_games', 'pct_points']]

In [12]:
df_nemesis_all.to_csv('../data/team_nemesis_all.csv')